# Showcase of functionality

##  Loading Data

## NWB

In [1]:
from pathlib import Path
from pynwb import NWBHDF5IO

stub_file = False 
nwb_folder_path = Path.home() / "conversion_nwb" 

if stub_file:
    nwb_folder_path = nwb_folder_path / "nwb_stub"
    assert nwb_folder_path.is_dir()

nwbfile_path = nwb_folder_path / "pico_20230215_161322.nwb"
#nwbfile_path = folder_path / "pico_20230214_140610.nwb"
nwbfile_path = nwb_folder_path / "pico_20230221_130510.nwb"
assert nwbfile_path.is_file(), f"{nwbfile_path} does not exist"


io = NWBHDF5IO(nwbfile_path, mode="r")
nwbfile = io.read()
nwbfile

,location,group,group_name,channel_name,spike_scope_voltage_thresh,native_order,rel_y,board_stream_num,electrode_impedance_phase,probe,chip_channel_num,group_electrode_number,rel_x,electrode_impedance_magnitude,channel_names,spike_scope_digital_edge_polarity,channel_ids,custom_order,spike_scope_digital_trigger_channel,spike_scope_trigger_mode
id,,,,,,,,,,,,,,,,,,,,
0,unknown,ABC pynwb.ecephys.ElectrodeGroup at 0x138332296405264\nFields:\n description: Group ABC electrodes.\n device: Intan pynwb.device.Device at 0x138332296426000\nFields:\n description: Intan recording\n manufacturer: Intan\n\n,ABC,A-000,-70.0,0.0,3200.0,0.0,-16.892567,ProbeABC,0.0,0.0,3600.0,24895.191406,A-000,0.0,A-000,0.0,0.0,1.0
1,unknown,ABC pynwb.ecephys.ElectrodeGroup at 0x138332296405264\nFields:\n description: Group ABC electrodes.\n device: Intan pynwb.device.Device at 0x138332296426000\nFields:\n description: Intan recording\n manufacturer: Intan\n\n,ABC,A-001,-70.0,1.0,2800.0,0.0,-21.669025,ProbeABC,1.0,1.0,3200.0,16327.307617,A-001,0.0,A-001,1.0,0.0,1.0
2,unknown,ABC pynwb.ecephys.ElectrodeGroup at 0x138332296405264\nFields:\n description: Group ABC electrodes.\n device: Intan pynwb.device.Device at 0x138332296426000\nFields:\n description: Intan recording\n manufacturer: Intan\n\n,ABC,A-002,-70.0,2.0,2800.0,0.0,-20.287891,ProbeABC,2.0,2.0,3600.0,18088.191406,A-002,0.0,A-002,2.0,0.0,1.0
3,unknown,ABC pynwb.ecephys.ElectrodeGroup at 0x138332296405264\nFields:\n description: Group ABC electrodes.\n device: Intan pynwb.device.Device at 0x138332296426000\nFields:\n description: Intan recording\n manufacturer: Intan\n\n,ABC,A-003,-70.0,3.0,2400.0,0.0,-21.501488,ProbeABC,3.0,3.0,3600.0,10623.975586,A-003,0.0,A-003,3.0,0.0,1.0


### Display the stimuli

In [ ]:
an_image = nwbfile.stimulus["stimuli"].images["im0"]

an_image_data = an_image.data[:]

import matplotlib.pyplot as plt

plt.imshow(an_image_data)

### Amplifiers

This is an example of how to load data.

In [ ]:
from pathlib import Path
from spikeinterface.extractors import IntanRecordingExtractor
from dicarlo_lab_to_nwb.conversion.data_locator import locate_intan_file_path

data_folder = Path("/media/heberto/One Touch/DiCarlo-CN-data-share")
image_set_name = "domain-transfer-2023"
subject = "pico"
session_date = "20230214"
session_time = "140610"

intan_file_path = locate_intan_file_path(
    data_folder=data_folder,
    image_set_name=image_set_name,
    subject=subject,
    session_date=session_date,
    session_time=session_time,
)

recording = IntanRecordingExtractor(
    file_path=intan_file_path,
    stream_name="RHD2000 amplifier channel",
    all_annotations=True,
    ignore_integrity_checks=False,
)
recording

This particular example has timestamps discontinuities, to load the data regardless we set the parameter `ignore_integrity_checks=True`.

In [ ]:
recording = IntanRecordingExtractor(
    file_path=intan_file_path,
    stream_name="RHD2000 amplifier channel",
    all_annotations=True,
    ignore_integrity_checks=True,
)
recording

### Auxiliary input

In [ ]:
recording_auxiliary_input = IntanRecordingExtractor(
    file_path=intan_file_path,
    stream_name="RHD2000 auxiliary input channel",
    all_annotations=True,
    ignore_integrity_checks=True,
)

recording_auxiliary_input

### ADC input

In [ ]:
recording_adc_input = IntanRecordingExtractor(
    file_path=intan_file_path,
    stream_name="USB board ADC input channel",
    all_annotations=True,
    ignore_integrity_checks=True,
)

recording_adc_input

### Digital channel 
Requires neo version from github https://github.com/NeuralEnsemble/python-neo/

In [ ]:
recording_digital = IntanRecordingExtractor(
    file_path=intan_file_path,
    stream_name="USB board digital input channel",
    all_annotations=True,
    ignore_integrity_checks=True,
)

recording_digital

## Loading the probe

In [ ]:
from dicarlo_lab_to_nwb.conversion.probe import build_probe_group
from dicarlo_lab_to_nwb.conversion.data_locator import locate_intan_file_path
from spikeinterface.extractors import IntanRecordingExtractor

data_folder = Path("/media/heberto/One Touch/DiCarlo-CN-data-share")
image_set_name = "domain-transfer-2023"
subject = "pico"
session_date = "20230214"
session_time = "140610"


intan_file_path = locate_intan_file_path(
    data_folder=data_folder,
    image_set_name=image_set_name,
    subject=subject,
    session_date=session_date,
    session_time=session_time,
)


stream_name = "RHD2000 amplifier channel"
recording = IntanRecordingExtractor(
    file_path=intan_file_path,
    stream_name=stream_name,
    ignore_integrity_checks=True,
    all_annotations=True,
)


probe_group = build_probe_group(recording=recording)


from probeinterface.plotting import plot_probe
import matplotlib.pyplot as plt
import numpy as np



fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(111)

probe = probe_group.probes[0]
channel_ids = recording.get_channel_ids()
corresponding_channel_ids = [channel_ids[i] for i in probe.device_channel_indices]

text_on_contact = np.asarray(corresponding_channel_ids)

plot_probe(probe=probe, ax=ax, with_contact_id=True, text_on_contact=text_on_contact)

In [ ]:
from probeinterface.plotting import plot_probe_group

fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(111)


plot_probe_group(probe_group, ax=ax, same_axes=True, with_contact_id=False)


# Sorting Pipeline

To run a sorting pipeline we need a recording with a geometry attached.

In [ ]:
from spikeinterface.extractors import IntanRecordingExtractor
from spikeinterface.sorters import run_sorter_by_property


from dicarlo_lab_to_nwb.conversion.data_locator import locate_intan_file_path
from dicarlo_lab_to_nwb.conversion.probe import attach_probe_to_recording

data_folder = Path("/media/heberto/One Touch/DiCarlo-CN-data-share")
image_set_name = "domain-transfer-2023"
subject = "pico"
session_date = "20230214"
session_time = "140610"


intan_file_path = locate_intan_file_path(
    data_folder=data_folder,
    image_set_name=image_set_name,
    subject=subject,
    session_date=session_date,
    session_time=session_time,
)


stream_name = "RHD2000 amplifier channel"
recording = IntanRecordingExtractor(
    file_path=intan_file_path,
    stream_name=stream_name,
    ignore_integrity_checks=True,
    all_annotations=True,
)


attach_probe_to_recording(recording=recording)
recording

Most sorters have been designed with high density probes in mind. They will work with a single channel probe, but the results may not be as good as some units might be supressed by the spatial regularization.

Because of this we performed sorting in two ways so you can compare the results:

1. We do one sorting per probe
2. We do one sorting per channel to avoid interference of the spatial regularization



## Performing a sorting per probe

In [ ]:
from spikeinterface.core import load_extractor

sorting_folder = Path("./sorting_done")
overwrite = False

if sorting_folder.exists() and not overwrite:
    sorting = load_extractor(sorting_folder)
else:
    sorting = run_sorter_by_property(
        sorter_name="kilosort2",
        recording=recording,
        folder="./sorting_folder_probe",
        grouping_property="probe",
        docker_image=True,
    )

    sorting.save(folder=sorting_folder)
    

Action item:
* How to save the sorting results to numpy
* Quality metrics:
    * Which channels are visually driven? we repeat the image 20 times, we randomly choose the first 10 images of the set same of itme
and correlate the responses.
    *   

In [ ]:
sorting

In [ ]:
from spikeinterface.core import create_sorting_analyzer


sorting_analyzer = create_sorting_analyzer(sorting=sorting, recording=recording)




## Performing a sorting per channel

In [ ]:
sorting = run_sorter_by_property(
    sorter_name="kilosort3",
    recording=recording,
    folder="./sorting_folder_per_channel",
    grouping_property="channel_names",
    docker_image=True,
)


In [ ]:
from spikeinterface.sorters import available_sorters

available_sorters()

In [ ]:
from spikeinterface.core import load_extractor

sorting_folder = Path("./sorting_done_per_channel")
overwrite = False

if sorting_folder.exists() and not overwrite:
    sorting = load_extractor(sorting_folder)
else:
    sorting = run_sorter_by_property(
        sorter_name="tridesclous",
        recording=recording,
        folder="./sorting_folder_per_channel",
        grouping_property="channel_names",
        docker_image=True,
    )

    sorting.save(folder=sorting_folder)
    


# Peak Detection Pipeline

## Artificial data

In [ ]:
import spikeinterface.widgets as sw

from spikeinterface.core.generate import generate_ground_truth_recording


recording, sorting = generate_ground_truth_recording(num_channels=4, num_units=1, durations=[1], seed=0)


w_ts = sw.plot_traces(recording, time_range=(0, 1))
w_rs = sw.plot_rasters(sorting, time_range=(0, 1))

In [ ]:
import numpy as np
from dicarlo_lab_to_nwb.conversion.pipeline import di_carlo_peak_detection



job_kwargs = dict(n_jobs=1, verbose=True, progress_bar=True, chunk_duration=1.0)
noise_threshold = 3  # The number of standard deviations for peak detection

spike_times_per_channel = di_carlo_peak_detection(recording=recording, noise_threshold=noise_threshold, job_kwargs=job_kwargs)

In [ ]:
sorting.get_unit_spike_train(0, return_times=True)

In [ ]:
spike_times_per_channel[0]

## Your data

In [ ]:
from pathlib import Path

import spikeinterface.widgets as sw
from dicarlo_lab_to_nwb.conversion.data_locator import locate_intan_file_path

from spikeinterface.extractors import IntanRecordingExtractor


data_folder = Path("/media/heberto/One Touch/DiCarlo-CN-data-share")
image_set_name = "domain-transfer-2023"
subject = "pico"
session_date = "20230214"
session_time = "140610"


intan_file_path = locate_intan_file_path(
    data_folder=data_folder,
    image_set_name=image_set_name,
    subject=subject,
    session_date=session_date,
    session_time=session_time,
)



recording = IntanRecordingExtractor(
    file_path=intan_file_path,
    stream_name="RHD2000 amplifier channel",
    all_annotations=True,
    ignore_integrity_checks=True,
)

# If you want to select only one channel
channel_ids = recording.get_channel_ids()[0:1]
recording = recording.select_channels(channel_ids=channel_ids)
w_ts = sw.plot_traces(recording, time_range=(0, 1), return_scaled=True)


#### Preprocess

In [ ]:
from dicarlo_lab_to_nwb.conversion.pipeline import DiCarloBandPass, DiCarloNotch


f_notch = 60  # Hz
bandwidth = 10
f_low = 300.0
f_high = 6000.0

vectorized = True 
notched_recording = DiCarloNotch(recording, f_notch=f_notch, bandwidth=bandwidth, vectorized=vectorized)
preprocessed_recording = DiCarloBandPass(notched_recording, f_low=f_low, f_high=f_high, vectorized=vectorized)

# For this instance each array 96 channels, 400 micrometes apart
w_ts = sw.plot_traces(preprocessed_recording, time_range=(0, 1), return_scaled=True)

#### Run the peak detection on a short portion of the data

In [ ]:
from dicarlo_lab_to_nwb.conversion.pipeline import di_carlo_peak_detection

noise_threshold = 3  # The number of standard deviations for peak detection

start_frame = 0
seconds_of_data = 1.0
end_frame = int(preprocessed_recording.sampling_frequency * seconds_of_data)
preprocessed_recording = preprocessed_recording.frame_slice(start_frame=start_frame, end_frame=end_frame)

spike_times_per_channel = di_carlo_peak_detection(
    recording=preprocessed_recording,
    noise_threshold=noise_threshold,
)

spike_times_per_channel

#### Everything can be wrapped up in a couple of lines

In [ ]:
from spikeinterface.extractors import IntanRecordingExtractor
from dicarlo_lab_to_nwb.conversion.pipeline import thresholding_pipeline
from dicarlo_lab_to_nwb.conversion.data_locator import locate_intan_file_path


image_set_name = "domain-transfer-2023"
subject = "pico"
session_date = "20230214"
session_time = "140610"

# Parameters of the pipeline
f_notch = 60  # Hz
bandwidth = 10
f_low = 300.0
f_high = 6000.0
noise_threshold = 3  # The number of standard deviations for peak detection


data_folder = Path("/media/heberto/One Touch/DiCarlo-CN-data-share")

intan_file_path = locate_intan_file_path(
    data_folder=data_folder,
    image_set_name=image_set_name,
    subject=subject,
    session_date=session_date,
    session_time=session_time,
)


stream_name = "RHD2000 amplifier channel"
recording = IntanRecordingExtractor(
    file_path=intan_file_path,
    stream_name=stream_name,
    ignore_integrity_checks=True,
    all_annotations=True,
)

spike_times_per_channel_vectorized = thresholding_pipeline(
    recording=recording.frame_slice(start_frame=0, end_frame=1000), # Remove frame_slice to run the whole pipeline
    f_notch=f_notch,
    bandwidth=bandwidth,
    f_low=f_low,
    f_high=f_high,
    noise_threshold=noise_threshold,
)

spike_times_per_channel

# Calculating PSTH

In [1]:
%load_ext autoreload
%autoreload 
from pathlib import Path

from spikeinterface.extractors import IntanRecordingExtractor
from dicarlo_lab_to_nwb.conversion.pipeline import thresholding_pipeline
from dicarlo_lab_to_nwb.conversion.data_locator import locate_intan_file_path
from dicarlo_lab_to_nwb.conversion.probe import attach_probe_to_recording

data_folder = Path("/media/heberto/One Touch/DiCarlo-CN-data-share")
image_set_name = "domain-transfer-2023"
subject = "pico"
session_date = "20230214"
session_time = "140610"

# Parameters of the pipeline
f_notch = 60  # Hz
bandwidth = 10
f_low = 300.0
f_high = 6000.0
noise_threshold = 3  # The number of standard deviations for peak detection

intan_file_path = locate_intan_file_path(
    data_folder=data_folder,
    image_set_name=image_set_name,
    subject=subject,
    session_date=session_date,
    session_time=session_time,
)


stream_name = "RHD2000 amplifier channel"
recording = IntanRecordingExtractor(
    file_path=intan_file_path,
    stream_name=stream_name,
    ignore_integrity_checks=True,
    all_annotations=True,
)


attach_probe_to_recording(recording=recording)
chunk_duration = 10.0  # 10 seconds
job_kwargs = dict(n_jobs=-1, progress_bar=True, verbose=True, chunk_duration=chunk_duration)

dict_of_recordings = recording.split_by(property="probe", outputs="dict")
dict_of_spikes_times_per_channel = {}

for probe_name, recording in dict_of_recordings.items():
    spikes_times_per_channel = thresholding_pipeline(
        recording=recording,
        f_notch=f_notch,
        bandwidth=bandwidth,
        f_low=f_low,
        f_high=f_high,
        noise_threshold=noise_threshold,
        vectorized=True,
        job_kwargs=job_kwargs,
    )
    
    dict_of_spikes_times_per_channel[probe_name] = spikes_times_per_channel

# We merge all the dictionaries
dict_of_spikes_times = {key: value for d in dict_of_spikes_times_per_channel.values() for key, value in d.items()}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
DiCarloPeakDetectionPipeline 
n_jobs=16 - samples_per_chunk=200,000 - chunk_memory=73.24 MiB - total_memory=1.14 GiB - chunk_duration=10.00s


DiCarloPeakDetectionPipeline:   0%|          | 0/248 [00:00<?, ?it/s]

DiCarloPeakDetectionPipeline 
n_jobs=16 - samples_per_chunk=200,000 - chunk_memory=73.24 MiB - total_memory=1.14 GiB - chunk_duration=10.00s


DiCarloPeakDetectionPipeline:   0%|          | 0/248 [00:00<?, ?it/s]

In [2]:
import pandas as pd
from dicarlo_lab_to_nwb.conversion.data_locator import locate_mworks_processed_file_path

mworks_processed_file_path = locate_mworks_processed_file_path(
    data_folder=data_folder,
    image_set_name=image_set_name,
    subject=subject,
    session_date=session_date,
    session_time=session_time,
)


mworks_processed_file_path = Path(mworks_processed_file_path)
dtype = {"stimulus_presented": np.uint32, "fixation_correct": bool}
mwkorks_df = pd.read_csv(mworks_processed_file_path, dtype=dtype)
ground_truth_time_column = "samp_on_us"
stimuli_presentation_times_seconds = mwkorks_df[ground_truth_time_column] / 1e6
stimuli_presentation_id = mwkorks_df["stimulus_presented"]
all_stimuli = stimuli_presentation_id.unique()

stimuli_presentation_times_dict = {
    stimulus_id: stimuli_presentation_times_seconds[stimuli_presentation_id == stimulus_id].values for stimulus_id in all_stimuli
}

spike_times_list = [spike_times for spike_times in dict_of_spikes_times.values()]

In [10]:
from dicarlo_lab_to_nwb.conversion.psth import calculate_event_psth

# Make 10 bins of 0.200 seconds each
number_of_bins = 10
bin_width_in_milliseconds = 400.0 / number_of_bins
#This means the first bin starts 200 ms before the image presentation
milliseconds_from_event_to_first_bin = -200.0  # 
max_repetitions = stimuli_presentation_id.value_counts().max()

# Let's calculate the PSTH for a single stimuli
a_stimuli = all_stimuli[0]
stimulus_presentation_times = stimuli_presentation_times_dict[a_stimuli]
psth_per_stimuli = calculate_event_psth(
    spike_times_list=spike_times_list,
    event_times_seconds=stimulus_presentation_times,
    bin_width_in_milliseconds=bin_width_in_milliseconds,
    number_of_bins = number_of_bins,
    milliseconds_from_event_to_first_bin=milliseconds_from_event_to_first_bin,
    number_of_events=max_repetitions,
)

psth_per_stimuli[1, ...]

array([[ 0.,  2.,  0.,  3.,  0.,  0.,  1.,  0.,  1.,  5.],
       [ 0.,  1.,  2.,  2.,  7.,  8.,  5., 13., 11.,  0.],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]])

In [11]:
from dicarlo_lab_to_nwb.conversion.psth import calculate_event_psth

# Make 10 bins of 0.200 seconds each
number_of_bins = 10
bin_width_in_milliseconds = 400.0 / number_of_bins
#This means the first bin starts 200 ms before the image presentation
milliseconds_from_event_to_first_bin = -200.0  # 
max_repetitions = stimuli_presentation_id.value_counts().max()

# Let's calculate the PSTH for a single stimuli
a_stimuli = all_stimuli[0]
stimulus_presentation_times = stimuli_presentation_times_dict[a_stimuli]
psth_per_stimuli = calculate_event_psth(
    spike_times_list=spike_times_list,
    event_times_seconds=stimulus_presentation_times,
    bin_width_in_milliseconds=bin_width_in_milliseconds,
    number_of_bins = number_of_bins,
    milliseconds_from_event_to_first_bin=milliseconds_from_event_to_first_bin,
    number_of_events=max_repetitions,
)

psth_per_stimuli[1, ...]

array([[ 0.,  2.,  0.,  3.,  0.,  0.,  1.,  0.,  1.,  5.],
       [ 0.,  1.,  2.,  2.,  7.,  8.,  5., 13., 11.,  0.],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]])

### Agregate psth for all stimuli in session

In [12]:
from tqdm.auto import tqdm
from dicarlo_lab_to_nwb.conversion.psth import calculate_event_psth
import time

number_of_units = len(spike_times_list)
number_of_stimuli = len(stimuli_presentation_times_dict)

session_psth = np.full(
    shape=(number_of_units, number_of_stimuli, max_repetitions, number_of_bins), fill_value=np.nan
)
psth_per_stimuli_dict = {}

time_start = time.time()
for stimulus_index, (stimulus_id, stimulus_times) in enumerate(tqdm(stimuli_presentation_times_dict.items(), desc="Processing Stimuli")):
    psth_per_stimuli = calculate_event_psth(
        spike_times_list=spike_times_list,
        event_times_seconds=stimulus_times,  # make sure this is correct
        bin_width_in_milliseconds=bin_width_in_milliseconds,
        number_of_bins=number_of_bins,
        milliseconds_from_event_to_first_bin=milliseconds_from_event_to_first_bin,
        number_of_events=max_repetitions,
    )
    session_psth[:, stimulus_index, :, :] = psth_per_stimuli
    
session_psth_numba = session_psth

time_stop = time.time()
time_numba = time_stop - time_start
print(f"Time elapsed: {time_numba:2.2f} seconds")

Processing Stimuli: 100%|██████████| 3138/3138 [00:35<00:00, 87.57it/s] 

Time elapsed: 35.84 seconds


To compare to a baseline this is the naive numpy implementation

In [ ]:
from tqdm import tqdm
from dicarlo_lab_to_nwb.conversion.psth import calculate_event_psth_numpy_naive
import time 

number_of_units = len(spike_times_list)
number_of_stimuli = len(stimuli_presentation_times_dict)

session_psth = np.full(
    shape=(number_of_units, number_of_stimuli, max_repetitions, number_of_bins), fill_value=np.nan
)

start = time.time()
for stimulus_index, (stimulus_id, stimulus_times) in enumerate(tqdm(stimuli_presentation_times_dict.items(), desc="Processing Stimuli")):
    psth_per_stimuli = calculate_event_psth_numpy_naive(
        spike_times_list=spike_times_list,
        event_times_seconds=stimulus_times,  # make sure this is correct
        bin_width_in_milliseconds=bin_width_in_milliseconds,
        number_of_bins=number_of_bins,
        milliseconds_from_event_to_first_bin=milliseconds_from_event_to_first_bin,
        number_of_events=max_repetitions,
    )
    session_psth[:, stimulus_index, :, :] = psth_per_stimuli
    
session_psth_naive = session_psth

time_stop = time.time()
time_naive = time_stop - time_start
print(f"Time elapsed: {time_naive:2.2f} seconds")

In [ ]:
np.allclose(session_psth_numba, session_psth_naive, equal_nan=True)

On my local machine this is ~10x faster than the naive numpy implementation